In [ ]:
import pygame
import random
import matplotlib.pyplot as plt
import pickle 
from PIL import Image  

pygame.init()

# colors
black = (0, 0, 0)
white = (255, 255, 255)
background_color = (255, 255, 240)  # Light Ivory
grid_line_color = (211, 211, 211)   # Light Gray for grid lines

pastel_colors = [
    (0, 178, 89),     # Bright Mint Green
    (140, 0, 255),    # Vivid Lavender
    (255, 94, 77),    # Vivid Peach
    (255, 204, 0),    # Bright Yellow
    (0, 102, 204),    # Bright Sky Blue
    (255, 63, 63),    # Bright Coral
    (0, 102, 153),    # Deep Powder Blue
    (128, 0, 128)     # Deep Thistle
]



# resolution
x, y = 1000, 1000

tile_size = 10
width = x // tile_size
height = y // tile_size
fps = 1000  

screen = pygame.display.set_mode((x, y))

clock = pygame.time.Clock()

# Function to generate random positions
def gen(num):
    return set([((random.randrange(0, height), random.randrange(0, width)), random.choice(pastel_colors)) for _ in range(num * tile_size)])

# Function to draw grid
def draw_grid(positions):
    for position, color in positions:
        # unpacking the position
        col, row = position
        top_left = (col * tile_size, row * tile_size)
        pygame.draw.rect(screen, color, (*top_left, tile_size, tile_size))

    for row in range(height):
        pygame.draw.line(screen, grid_line_color, (0, row * tile_size), (x, row * tile_size))

    for col in range(width):
        pygame.draw.line(screen, grid_line_color, (col * tile_size, 0), (col * tile_size, y))

# Function to update the grid
def adjust_grid(positions):
    all_neighbours = set()
    new_positions = set()

    # Convert positions to a dictionary ( lookup table )
    pos_dict = {pos: color for pos, color in positions}

    for position, color in positions:
        neighbours = get_neighbours(position)
        all_neighbours.update(neighbours)

        neighbours = list(filter(lambda x: x in pos_dict, neighbours))
        if len(neighbours) in [2, 3]:
            new_positions.add((position, color))

    for position in all_neighbours:  # neighbours of neighbours
        neighbours = get_neighbours(position)

        # Filter to only live neighbors
        live_neighbours = list(filter(lambda x: x in pos_dict, neighbours))

        if len(live_neighbours) == 3:
            # Randomly inherit color from live neighbors
            inherited_color = random_inheritance([pos_dict[n] for n in live_neighbours])
            new_positions.add((position, inherited_color))

    return new_positions

# Function for random inheritance of colors
def random_inheritance(colors):
    # Select two random colors from the neighbors
    chosen_colors = random.sample(colors, 2)
    
    # Blend the two colors by averaging their RGB values
    r = (chosen_colors[0][0] + chosen_colors[1][0]) // 2
    g = (chosen_colors[0][1] + chosen_colors[1][1]) // 2
    b = (chosen_colors[0][2] + chosen_colors[1][2]) // 2
    
    return (r, g, b)

# Function to get neighbors of a position
def get_neighbours(pos):
    a, b = pos
    neighbours = []
    for da in [-1, 0, 1]:
        if a + da < 0 or a + da > width:
            continue
        for db in [-1, 0, 1]:
            if b + db < 0 or b + db > height:
                continue
            if da == 0 and db == 0:
                continue
            neighbours.append((a + da, b + db))
    return neighbours

# Function to save the game screen as an image
def save_screenshot(filename="screenshot.png"):
    pygame.image.save(screen, filename)
    print(f"Screenshot saved as {filename}")

# Function to save the plot as an image
def save_plot(live_cells_per_gen, filename="plot.png"):
    plt.plot(live_cells_per_gen)
    plt.title('Number of Live Cells vs Generations')
    plt.xlabel('Generation')
    plt.ylabel('Number of Live Cells')
    plt.savefig(filename)
    plt.close()  # Close the plot after saving
    print(f"Plot saved as {filename}")

# Function to save the game state (positions of live cells)
def save_game_state(positions, filename="game_state.pkl"):
    with open(filename, 'wb') as f:
        pickle.dump(positions, f)
    print(f"Game state saved as {filename}")

# Function to create GIF from screenshots
def create_gif(screenshots, gif_filename="game_animation.gif"):
    frames = [Image.open(img) for img in screenshots]
    frame_one = frames[0]
    frame_one.save(gif_filename, format="GIF", append_images=frames[1:], save_all=True, duration=100, loop=0)
    print(f"GIF saved as {gif_filename}")

# Main loop
def main():
    running = True
    playing = False

    count = 0
    update_freq = 120

    positions = set()
    live_cells_per_gen = []  # List to store the number of live cells per generation
    screenshots = []  # List to store screenshot filenames for the GIF

    while running:
        clock.tick(fps)

        if playing:
            count += 1

        if count >= update_freq:
            count = 0
            positions = adjust_grid(positions)
            live_cells_per_gen.append(len(positions))  # Track the number of live cells

            # Save screenshots for each generation
            screenshot_filename = f"screenshot_gen_{len(live_cells_per_gen)}.png"
            save_screenshot(screenshot_filename)
            screenshots.append(screenshot_filename)

        pygame.display.set_caption("playing" if playing else "paused")

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if event.type == pygame.MOUSEBUTTONDOWN:
                a, b = pygame.mouse.get_pos()
                col = a // tile_size
                row = b // tile_size
                pos = (col, row)

                if any(p[0] == pos for p in positions):
                    positions = set(filter(lambda p: p[0] != pos, positions))
                else:
                    positions.add((pos, random.choice(pastel_colors)))

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    playing = not playing

                if event.key == pygame.K_c:
                    positions = set()
                    playing = False
                    count = 0

                if event.key == pygame.K_g:
                    positions = gen(random.randrange(2, 5) * width)

        screen.fill(background_color)
        draw_grid(positions)
        pygame.display.update()

    # After quitting the game, save the final plot and create a GIF
    if live_cells_per_gen:
        save_plot(live_cells_per_gen, "final_live_cells_plot.png")

    # Create GIF from the screenshots
    if screenshots:
        create_gif(screenshots, "game_animation.gif")

    pygame.quit()

if __name__ == "__main__":
    main()
